In [1]:
# import dependencies
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d  # noqa: F401
import numpy as np
import pandas as pd
import seaborn as sns
import time as time

from sklearn import preprocessing

import scipy
import scipy.cluster.hierarchy as shc
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, fclusterdata, ward, single

from sklearn import cluster, datasets, metrics
from sklearn.cluster import AgglomerativeClustering, SpectralClustering
from sklearn.mixture import GaussianMixture

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

!pip freeze > import_trade_data.txt

In [2]:
# enter spreadsheet name without extension:'MidCapLargeCap20240314A' or 'LargeMegaCapCompanies20241231A'
file = input("Enter Excel filename without extension: ")
# sheet = input("Enter sheet name: ")

file_path = "data/" + file + ".xlsx"

Enter Excel filename without extension:  LargeMegaCapCompanies20241231A


In [3]:
# Read the Excel file
excel_data = pd.ExcelFile(file_path)

# List all sheet names in the workbook
sheet_names = excel_data.sheet_names
print(f"Sheet Names: {sheet_names}")

# # Choose a specific sheet name (replace 'Sheet1' with the desired sheet name)
# selected_sheet_name = 'PRN_StandUpToCancer'

# # Read data from the selected sheet into a DataFrame
# df = pd.read_excel(excel_data, sheet_name=selected_sheet_name)

# # Display the DataFrame
# print(f"\nData from '{selected_sheet_name}':")
# print(df)

Sheet Names: ['Summary', 'Opinions', 'Price Performance', 'Fundamentals', 'Earnings & Dividends', 'Technicals']


In [4]:
# Choose a specific sheet name (replace 'Sheet1' with the desired sheet name)
selected_sheet_name0 = sheet_names[0]

# Read data from the selected sheet into a DataFrame
df = pd.read_excel(excel_data, sheet_name=selected_sheet_name0)

# Display the DataFrame
print(f"\nData from '{selected_sheet_name0}':")
print(df)


Data from 'Summary':
    Symbol                     Company Name  \
0      MMM                            3M Co   
1    ABBNY                    ABB Ltd (ADR)   
2      ABT              Abbott Laboratories   
3     ABBV                       AbbVie Inc   
4      ACN                    Accenture Plc   
..     ...                              ...   
979    ZTS                       Zoetis Inc   
980     ZM          Zoom Communications Inc   
981     ZS                      Zscaler Inc   
982    ZTO   ZTO Express (Cayman) Inc (ADR)   
983  ZURVY  Zurich Insurance Group AG (ADR)   

                              Industry Prior Close 5 Day Chg 4 Week Chg  \
0         Consumer Goods Conglomerates      131.18    -0.12%      1.00%   
1    Machinery, Equipment & Components       55.24    -0.79%     -3.00%   
2      Healthcare Equipment & Supplies      115.27    -1.25%     -3.00%   
3                      Pharmaceuticals      179.20     0.35%     -1.00%   
4               Software & IT Services

In [5]:
# Choose a specific sheet name (replace 'Sheet1' with the desired sheet name)
selected_sheet_name0 = sheet_names[0]

# Read data from the selected sheet into a DataFrame
df = pd.read_excel(excel_data, sheet_name = selected_sheet_name0)

# getting length of list
length = len(sheet_names)
  
# Iterate through the list to merge the data
for i in range(1, length):
    # Choose a specific sheet name (replace 'Sheet1' with the desired sheet name)
    selected_sheet_namei = sheet_names[i]
    # Read data from the selected sheet into a DataFrame
    dfi = pd.read_excel(excel_data, sheet_name = selected_sheet_namei)
    dfi = dfi.drop(['Symbol'], axis=1)
    df = pd.merge(df, dfi, on = 'Company Name', how = 'left')
#     df = pd.concat([df,dfi], ignore_index=True, axis=1)
#     print(df)
#     df_new = df.merge(dfi,how='left', left_on='Company Name', right_on='Company Name')
    

In [6]:
df

,Symbol,Company Name,Industry,Prior Close,5 Day Chg_x,4 Week Chg_x,52 Week Chg_x,Mkt Cap,Refinitiv,Argus Analyst,...,Revenue Growth CFY*,Revenue Growth NFY*,Dividend Yield,Dividend Growth 5 Year,52 Week Chg_y,Vs. 13-Day EMA*,Vs. 50-Day EMA*,% Below 52 Wk High,% Above 52 Wk Low,Short Interest as % Float
0,MMM,3M Co,Consumer Goods Conglomerates,131.18,-0.12%,1.00%,45.00%,$71.435B,Positive,Buy,...,-24.78%,2.85%,2.17%,1.98%,45.00%,-0.38%,-0.73%,8.64%,70.70%,1.11%
1,ABBNY,ABB Ltd (ADR),"Machinery, Equipment & Components",55.24,-0.79%,-3.00%,26.00%,$102.780B,--,--,...,2.04%,5.35%,1.77%,2.61%,26.00%,-2.68%,-4.03%,9.05%,34.33%,0.00%
2,ABT,Abbott Laboratories,Healthcare Equipment & Supplies,115.27,-1.25%,-3.00%,5.00%,$199.931B,Neutral,Buy,...,4.66%,7.12%,2.09%,12.74%,5.00%,-1.25%,-1.90%,7.27%,13.13%,0.70%
3,ABBV,AbbVie Inc,Pharmaceuticals,179.20,0.35%,-1.00%,13.00%,$316.671B,Neutral,Buy,...,3.21%,5.79%,3.72%,10.52%,13.00%,-0.22%,-2.27%,15.01%,14.73%,1.05%
4,ACN,Accenture Plc,Software & IT Services,360.43,-3.79%,2.00%,3.00%,$243.244B,Neutral,Buy,...,5.96%,6.73%,1.68%,11.31%,3.00%,-1.82%,-1.42%,9.04%,26.48%,0.87%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,ZTS,Zoetis Inc,Pharmaceuticals,165.52,-1.58%,-7.00%,-15.00%,$74.677B,Neutral,Buy,...,8.39%,4.92%,1.23%,24.37%,-15.00%,-3.40%,-7.98%,19.10%,12.04%,1.39%
980,ZM,Zoom Communications Inc,Software & IT Services,85.44,-3.35%,0.00%,18.00%,$22.394B,Neutral,Hold,...,0.00%,0.00%,0.00%,0.00%,18.00%,-1.92%,2.41%,10.85%,50.25%,3.27%
981,ZS,Zscaler Inc,Software & IT Services,187.63,-2.27%,-11.00%,-15.00%,$28.790B,Neutral,--,...,21.73%,20.64%,0.00%,0.00%,-15.00%,-4.23%,-6.16%,29.46%,19.34%,5.00%
982,ZTO,ZTO Express (Cayman) Inc (ADR),Freight & Logistics Services,19.91,-2.72%,3.00%,0.00%,$12.030B,--,--,...,12.41%,9.35%,5.03%,0.00%,0.00%,-1.51%,-6.49%,29.85%,21.36%,4.07%
